In [1]:
import os
import cv2
import PIL.Image as Image
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from enhanced_model import enhance_net
from denoise_model import denoising_net

In [2]:
model_name = ['Small_Denoised_sigmoid_LOL_Lambda2_1']
model_name = model_name[0]

mode = ['Large', 'Small']
mode = mode[1]

add_noise = False
print(model_name)

i = 'epoch1901_ssim0.869741_psnr33.173935'
denoising_model = denoising_net(input_shape=(None,None,3), mode=mode, model_name=model_name, train=False, add_noise=add_noise)
# model.enhancement_net.load_weights(enhancement_path)
denoising_path = './model/{0}/weights/{1}/'.format(model_name, i)
denoising_model.denoising_net.load_weights(denoising_path)

Small_Denoised_sigmoid_LOL_Lambda2_1


In [3]:
model_name = ['DCE', 'CSP_DCE', 'MSP_DCE', 'DCE++']
model_name = model_name[1]

print(model_name)


CSP_DCE


In [5]:
i = 80
for i in range(10,11,10):
    path = './model/{0}/weights/epoch{1}/'.format('CSP_DCE_Split2', i)
    # path = './model/CSP_DCE_1_25_10_1600/weights/epoch{0}/'.format(i)
    enhance_model = enhance_net(input_shape=(None,None,3), model_name=model_name)
    enhance_model.enhancement_net.load_weights(path)

    img_dir = './Dataset/low/'
    # img_dir = './Dataset/Face_Detection/DarkFace/'
    # img_dir = 'C://Users/user/Downloads/mh/'
    output_dir = './result/{0}/epoch{1}/'.format('CSP_DCE_Split2+LWDSN', i)
    # output_dir = './result/Face_Detection/'

    folder = os.path.exists(output_dir)
    if not folder:
        os.makedirs(output_dir)


    for i in range(10):
        strat = time.time()
        for file in os.listdir(img_dir):
            img_path = img_dir + file
            
            img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
            
            h,w,c = np.shape(img)
            img = tf.reshape(img, (1,h,w,c))
            img = tf.cast(img, dtype=tf.float32)
            
            image, parameter_map = enhance_model.predict(img)
            image,noise_img = denoising_model.predict(image)
            
            # denoising_image = NormalizeData(enhancemnet_image)
            image = np.reshape(image, (h,w,c))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # cv2.imwrite(output_dir +file, image*255)

        end = time.time()
        per_image_time = (end - strat)/len(os.listdir(img_dir))
        print(per_image_time)

0.10459090868631998
0.0935232957204183
0.09400480588277181
0.09423425992329916
0.09419201215108236
0.09278720219930013
0.09265246391296386
0.09323670069376627
0.09330388704935709
0.09299894968668619
